### 导入函数库


In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import jieba
import re

### 读取数据

In [2]:
train = pd.read_csv("./data/train.csv", sep="###__###",header = None)
test = pd.read_csv("./data/test.csv", sep="###__###",header = None)    

### 观察数据

In [8]:
train.columns= ['ID', 'Age', 'Gender', 'Education', 'Query_List']
train.head()

,ID,Age,Gender,Education,Query_List
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只..."
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...


In [9]:
test.columns= ['ID','Query_List']
test.head()

,ID,Query_List
0,ED89D43B9F602F96D96C25255F7C228C,陈学冬将出的作品\t刘昊然与谭松韵\t211学校的分数线\t谁唱的味道好听\t吻戏是真吻还是...
1,83C3B7B4AAF8074655A8079F561A76D6,e的0.0052次方\tqq怎么快速提现\t绝色倾城飞烟\t马克思主义基本原理概论\t康世恩...
2,CA9F675A024FB2353849350A35CF8B0F,黑暗文\tlpl夏季赛\t大富豪电玩城\t英雄联盟之电竞称王\t手机怎么扫描手机上的二维码\...
3,DE45B5C4E57AAEBCF3FDFA2A774093BF,中秋水库钓鱼\t鱼竿\t用蚯蚓钓鱼怎样调漂\t传统钓\t3号鱼钩\t鲫鱼汤的做法大全\t鱼饵...
4,406A681FB3DF81EC0E561796AE50AE50,号码吉凶\t退休干部死后配偶\t郫县有哪些大学\t胜利油田属于中石化还是中石油\t苏珊米勒狮...


### 数据介绍

本数据来源于真实搜索引擎数据，ID经过加密，训练集中人口属性数据存在部分未知的情况。数据所有字段如下表所示：

| 字段       | 说明                                                         |
| ---------- | ------------------------------------------------------------ |
| ID         | 加密后的ID                                                   |
| Age        | 0：未知年龄; 1：0-18岁; 2：19-23岁; 3：24-30岁; 4：31-40岁; 5：41-50岁; 6： 51-999岁 |
| Gender     | 0：未知1：男性2：女性                                        |
| Education  | 0：未知学历; 1：博士; 2：硕士; 3：大学生; 4：高中; 5：初中; 6：小学 |
| Query List | 搜索词列表                                                   |


### 查看是否有缺失值

In [101]:
print(train.info(),test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
ID            100000 non-null object
Age           100000 non-null int64
Gender        100000 non-null int64
Education     100000 non-null int64
Query_List    100000 non-null object
dtypes: int64(3), object(2)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
ID            100000 non-null object
Query_List    100000 non-null object
dtypes: object(2)
memory usage: 1.5+ MB
None None


数据中没有缺失值

### 观察文本数据

通过观察文本数据看是否存在需要手动处理的信息,在简单观察信息后，可以看出，不同的搜索记录以\t分隔，发现以下两种特殊情况： 

情况1 查询网址

In [36]:
train.iloc[0,4]

'柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1plpjtn9\t周公解梦大全查询2345\t曹云金再讽郭德纲\t总裁大人行行好\t中财网第一财经传媒\t教师节全文\t男子砸毁15墓碑\t黄岩岛最新填海图\t引起的疲\t缘来未迟落跑甜心不好惹\t梁朝伟与替身同框\t笑傲江湖电视剧任贤齐\t小起名字女孩名字\t海运行李到堪培拉\t确定\t诱爱99天 司少的天价宝贝\t什么是遥控魔棒\t徽信表情动态搞笑图片\t教师节征文100字\t安微联通网上营业厅\t甜宠百分百:校草的萌萌未婚妻\t豪门重生之暖爱成婚\tnikehypershift和kd5哪个好看\t韭菜炒鸡蛋\t陈赫玩王者荣耀\t虎牙楚河\t三国演义小说txt下载\t威县欧派\t炒馍花怎么做好吃\t黄岩岛最新消息2016年\t中秋节诗句大全祝福\t教师节征文\t菜谱\t柔和双沟卖的怎么样\t七位数开奖结果\t以色列停车场坍塌\t天龙家庭农场\t7.22什么星座\t新旧约圣经和合本下载\t4π\twifi万能钥匙\t威灵仙图片\t临泉长官天龙家庭农场\t早安总统大人\t百合\t莲藕的做法\t花街\t无锡\t蚬壳胃散怎么吃\t触手忆寒\t中秋节的诗句\t孟州电信 电子发票\t鸡丝汤的做法\t我等你\t临泉长官镇桥口李小刚农场\t朋仇\t全民k歌\t炸葱花\t蒜苔炒肉\t冰川的图片\tkd5\t…\t若风\t好奇纸尿裤\t清蒸鱼\t189.8是谁的平方\t重庆餐馆发生爆炸\t捡手机被失主抢劫\thttps://yunpan.cn/ocsqfgtfya2ewj\t炒馍花的家常做法\t三国演义小说百度云\t总裁掠爱小舅别太坏\t:https://yunpan.cn/cmh8tmeyraiww\t周公解梦\t查坦克冰川\t凉拌藕片的做法\t投票\t鸡丝炒什么好吃\t被时光掩埋的秘密小说下载\t中国电信电子发票\t张续豪\t关于月亮的诗句\t用酵母蒸馒头的方法\t赵丽颖碧瑶坐\t触手兵长\t图集 下载腾讯新闻,看街头混战武警\t厦门航空\t蚬壳胃散\t炒茄子做法\t身份类别怎么填\t最好的我们里面的方特在哪里\t牢里面的生活是怎样的\t强迫症有哪些表现\t白袍法师暖暖图片\t朋仇广场舞\t小宇热游\t蒸馒头的方法\t狡滑的意思\t黄石大冶东岳派出所服务电话\t三国演义小说下载txt\th

从这条数据中发现有时用户会对网址进行搜索，因此可考虑讲http://后面的网址提取出来进行替换

In [90]:
import re
from functools import reduce
pattern = re.compile('|'.join(['(http://)\w+(\..+?\t)','(https://)\w+(\..+?\t)']))
string = train.iloc[0,4]
for i in set(reduce(lambda x,y:x+y,pattern.findall(train.iloc[0,4]))):
    string = string.replace(i,'')
string

'柔和双沟\t女生\t中财网首页 财经\tpan周公解梦大全查询2345\t曹云金再讽郭德纲\t总裁大人行行好\t中财网第一财经传媒\t教师节全文\t男子砸毁15墓碑\t黄岩岛最新填海图\t引起的疲\t缘来未迟落跑甜心不好惹\t梁朝伟与替身同框\t笑傲江湖电视剧任贤齐\t小起名字女孩名字\t海运行李到堪培拉\t确定\t诱爱99天 司少的天价宝贝\t什么是遥控魔棒\t徽信表情动态搞笑图片\t教师节征文100字\t安微联通网上营业厅\t甜宠百分百:校草的萌萌未婚妻\t豪门重生之暖爱成婚\tnikehypershift和kd5哪个好看\t韭菜炒鸡蛋\t陈赫玩王者荣耀\t虎牙楚河\t三国演义小说txt下载\t威县欧派\t炒馍花怎么做好吃\t黄岩岛最新消息2016年\t中秋节诗句大全祝福\t教师节征文\t菜谱\t柔和双沟卖的怎么样\t七位数开奖结果\t以色列停车场坍塌\t天龙家庭农场\t7.22什么星座\t新旧约圣经和合本下载\t4π\twifi万能钥匙\t威灵仙图片\t临泉长官天龙家庭农场\t早安总统大人\t百合\t莲藕的做法\t花街\t无锡\t蚬壳胃散怎么吃\t触手忆寒\t中秋节的诗句\t孟州电信 电子发票\t鸡丝汤的做法\t我等你\t临泉长官镇桥口李小刚农场\t朋仇\t全民k歌\t炸葱花\t蒜苔炒肉\t冰川的图片\tkd5\t…\t若风\t好奇纸尿裤\t清蒸鱼\t189.8是谁的平方\t重庆餐馆发生爆炸\t捡手机被失主抢劫\tyunpan炒馍花的家常做法\t三国演义小说百度云\t总裁掠爱小舅别太坏\t:yunpan周公解梦\t查坦克冰川\t凉拌藕片的做法\t投票\t鸡丝炒什么好吃\t被时光掩埋的秘密小说下载\t中国电信电子发票\t张续豪\t关于月亮的诗句\t用酵母蒸馒头的方法\t赵丽颖碧瑶坐\t触手兵长\t图集 下载腾讯新闻,看街头混战武警\t厦门航空\t蚬壳胃散\t炒茄子做法\t身份类别怎么填\t最好的我们里面的方特在哪里\t牢里面的生活是怎样的\t强迫症有哪些表现\t白袍法师暖暖图片\t朋仇广场舞\t小宇热游\t蒸馒头的方法\t狡滑的意思\t黄石大冶东岳派出所服务电话\t三国演义小说下载txt\tzxjhjc9088松柏道馆\t10.1高速免费几天\t三国演义小说txt\t柔和双沟业务待遇\t酵母蒸馒头的方法\t初中家教一对一辅导\t口子窖\t中秋节祝福诗句\t侠岚

情况2：引用公式

In [37]:
train.iloc[100,4]

'小生梦惊魂\t鼠标怎么拆开图解\t我们的十年\t求导法则\t终极一班4百度云资源\t亲家过年\t电影\t小虎子闯世界\t六弄咖啡馆百度云\t一寸\t番12路公交车路线\t巴塞罗那\t微微一笑倾城电影网盘\t神厨电影资源分享\t法线方程\tpokemon go怎么玩不了\t广州北京路步行街\t终极一班4云盘资源\t窃资达人\t活色生香小说\t钢铁侠2\t钢铁侠3\tcosx周期\t封神传奇搜索百科\t钢铁侠4\t餐吧\t卡塔尔基地\t霸道总裁活色生香\t微波炉烤红薯\ty=x^3\t雪儿未删减版\tlimf(x)存在,则f(x)在x处\t星探电影\t安卓微信一键转发软件\t钟楚红雪儿未删减版\t地铁图\t猪仙\t心形线周长怎么求\t包皮过长会带来什么危害\t拐个皇帝回现代\t影音先锋看片资源\t猪八戒\ty=x^3+1是奇函数吗\tpokemon go进不去\t封神传奇有几部\t心形线\t香港片\t(2_x)sin(1/(2_x))\t番67全程\ttanx周期\t活色生香,灵书阅读\ttyrant美剧\tpokemon go怎么玩\t安西教练我想打篮球出自哪个漫画\t微微一笑很倾城电影百度云\t番68\t奇函数乘奇函数\t番12路线\t窃资达人资源\t封神传奇百度云\t93号航班\t电影百度云\t毛晓彤\t复仇者联盟\t联航93百度云\t李丽珍\tx→2,limax+b/(x-2)=2求ab\t三人行\t包皮环切术\t爱德华·诺顿\t神厨台湾资源分享\t番67全程时间\t神厨任容萱资源分享\t市桥到燕塘时间\t台湾神厨资源分享\t番禺区邮政编码\t神厨\t六弄咖啡馆\t窃资达人贴吧\t绿巨人\t童话村\t美国队长百度云\tlim(√(2x+1)-3)/(√(x-2)-√2)\t∩\t艾瑞克·巴纳\t朱宝意\t都市生活活色生香小说\t菊豆\t拐个皇帝回现代百度云\t93号航班百度云\t钟楚红雪儿百度云\t联航93\t深圳财付通科技有限公司\t王适娴谌龙照片\tsm\t对数\t活色生香广州小说\t罗京\t雪儿百度云\t复仇者联盟2\te∧1/x极限\t广东科技学院\t祈祷卓依婷\t微微一笑很倾城电影\t釜山行\t1-e∧(1/x)/(1+e∧(1/x))\tx→2,limax+b/(x-2)=2\t精灵宝贝复刻\t广东科技职业技术学院\t乔任梁\t孙悟空七打九尾狐\t

上面的文本存在公式，也应该想办法讲公式提取出来替换为统一的标记

In [68]:
import re
from functools import reduce
pattern = re.compile('\t(w+[\+\-]).*?\t')
string = train.iloc[100,4]
for i in set(reduce(lambda x,y:x+y,pattern.findall(train.iloc[100,4]))):
    string = string.replace(i,mathFomula)
string

TypeError: reduce() of empty sequence with no initial value

但是正则表达式还有问题

In [96]:
pattern = re.compile('\t\w+[=\+\^].*?\t')
pattern.findall(train.iloc[100,4])

['\ty=x^3\t', '\ty=x^3+1是奇函数吗\t']

In [55]:
pattern = re.compile('|'.join(['(http://)\w+(\..+?\t)','(https://)\w+(\..+?\t)']))
re=pattern.findall(train.iloc[0,4])


### 数据预处理

#### 读取停用词

In [3]:
stopwords = pd.read_csv('./data/stopwords.txt',sep='\t',encoding='utf-8',\
                       quoting=3,index_col=False,names=['stopword'])

没有加去停用词的程序

In [37]:
from functools import reduce
def preprocess(data):
    # delete website address,which can be refined later
    compiler = re.compile('\t')
    data = re.sub(compiler,'',data)
    #将网址用http://后首个字符串
    pattern = re.compile('|'.join(['(http://)\w+(\..+?\t)','(https://)\w+(\..+?\t)']))
    try:
        for i in set(reduce(lambda x,y:x+y,pattern.findall(data))):
            data = data.replace(i,'')
    except:
        pass
    data = jieba.lcut(data)
    # delete stopwords    
#     word_state = pd.DataFrame(data,columns=['segment'])
    # print(word_state['segment'].isin(stopwords.stopword).sum())
#     data = word_state[~word_state['segment'].isin(stopwords.stopword)]['segment'].tolist()
    return data


In [ ]:
from functools import reduce
def preprocess(data):
    # delete website address,which can be refined later
    compiler = re.compile('\t')
    data = re.sub(compiler,'',data)
    #将网址用http://后首个字符串
    pattern1 = re.compile('(http://)\w+(\..+?\t)')
    pattern2 = re.compile('(https://)\w+(\..+?\t)')
    try:
        for i in set(reduce(lambda x,y:x+y,pattern1.findall(data))):
            data = data.replace(i,'')
            print(data)        
    except Exception as e:
        try:
            for i in set(reduce(lambda x,y:x+y,pattern2.findall(data))):
                data = data.replace(i,'')
                print(data)
        except Exception as e:
            print(e)
    print(data)
    data = jieba.lcut(data)
    # delete stopwords    
#     word_state = pd.DataFrame(data,columns=['segment'])
#     # print(word_state['segment'].isin(stopwords.stopword).sum())
#     data = word_state[~word_state['segment'].isin(stopwords.stopword)]['segment'].tolist()
    return data

In [ ]:
import re
from functools import reduce
pattern = re.compile('|'.join(['(http://)\w+(\..+?\t)','(https://)\w+(\..+?\t)']))
string = train.iloc[0,4]
for i in set(reduce(lambda x,y:x+y,pattern.findall(train.iloc[0,4]))):
    string = string.replace(i,'')
string

#### 测试一下

In [21]:
train1 = train.iloc[:10,:]
train1.head()

,ID,Age,Gender,Education,Query_List
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只..."
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...


In [22]:
import time
time1 = time.time()
res = train1.loc[:,['Query_List']].applymap(preprocess)
time.time()-time1

柔和双沟女生中财网首页 财经http://pan.baidu.com/s/1plpjtn9周公解梦大全查询2345曹云金再讽郭德纲总裁大人行行好中财网第一财经传媒教师节全文男子砸毁15墓碑黄岩岛最新填海图引起的疲缘来未迟落跑甜心不好惹梁朝伟与替身同框笑傲江湖电视剧任贤齐小起名字女孩名字海运行李到堪培拉确定诱爱99天 司少的天价宝贝什么是遥控魔棒徽信表情动态搞笑图片教师节征文100字安微联通网上营业厅甜宠百分百:校草的萌萌未婚妻豪门重生之暖爱成婚nikehypershift和kd5哪个好看韭菜炒鸡蛋陈赫玩王者荣耀虎牙楚河三国演义小说txt下载威县欧派炒馍花怎么做好吃黄岩岛最新消息2016年中秋节诗句大全祝福教师节征文菜谱柔和双沟卖的怎么样七位数开奖结果以色列停车场坍塌天龙家庭农场7.22什么星座新旧约圣经和合本下载4πwifi万能钥匙威灵仙图片临泉长官天龙家庭农场早安总统大人百合莲藕的做法花街无锡蚬壳胃散怎么吃触手忆寒中秋节的诗句孟州电信 电子发票鸡丝汤的做法我等你临泉长官镇桥口李小刚农场朋仇全民k歌炸葱花蒜苔炒肉冰川的图片kd5…若风好奇纸尿裤清蒸鱼189.8是谁的平方重庆餐馆发生爆炸捡手机被失主抢劫https://yunpan.cn/ocsqfgtfya2ewj炒馍花的家常做法三国演义小说百度云总裁掠爱小舅别太坏:https://yunpan.cn/cmh8tmeyraiww周公解梦查坦克冰川凉拌藕片的做法投票鸡丝炒什么好吃被时光掩埋的秘密小说下载中国电信电子发票张续豪关于月亮的诗句用酵母蒸馒头的方法赵丽颖碧瑶坐触手兵长图集 下载腾讯新闻,看街头混战武警厦门航空蚬壳胃散炒茄子做法身份类别怎么填最好的我们里面的方特在哪里牢里面的生活是怎样的强迫症有哪些表现白袍法师暖暖图片朋仇广场舞小宇热游蒸馒头的方法狡滑的意思黄石大冶东岳派出所服务电话三国演义小说下载txthttp://zxjhjc9088.1688.com松柏道馆10.1高速免费几天三国演义小说txt柔和双沟业务待遇酵母蒸馒头的方法初中家教一对一辅导口子窖中秋节祝福诗句侠岚文王国窖42度价格表1001文王国窖42度价格表批注是什么意思殿下专属小丫头无锡爆炸炸茄子做法http://pan.baidu.com/s/1cor7gy大件行李邮寄烟火陈翔没想到,真没想到作文安徽滁州石坝镇虎牙小宇驾校培训跑长途骨质性关节炎左

bft20龙王传说长治到成都可以怎么坐火车龙王传说txt2 4 8 16老司机我恨双子座排第几名张家界旅游景点源代码索拉卡张艺兴歌曲到眉山东站的公交车金立s8换个摄像头多钱长治到眉山怎么坐车临汾到上海南站火车票瓦特是什么意思肉番漫画本子五代风影的阵容张家界有蹦极吗bft20手机雷主忍考90暗黑系头像四川冬天冷不冷降头术是真的吗金立s8摄像头多钱叶非夜肉番邪恶少女漫画本子女生左手中指戴戒指什么意思网名大全小样就是矫情是什么歌张艺兴忍考95水主太原到眉山临汾到上海的火车我恨双子座有多少万人双子座爱上一个人12345金立s8摄像头成都到眉山走不走成都东站金立s8分屏多任务太原到成都张家界旅游国内蹦极排名关晓彤风主忍考90里番肉番漫画本子12345什么意思12345代表什么意思张家界有蹦极的地方吗成都东到眉山汽车长治到重庆火车票张家界玻璃桥约会软件是真是假张艺兴mv关晓彤张艺兴吻戏关晓彤张艺兴火焰纹身太原到成都东少女漫画肉番本子库临汾到太原火车时刻表男人吃了痛经的药246810什么意思张家界旅游景点玻璃桥bycbbs源代码索拉卡特效长治到成都的火车怎么做欧米伽小队 幽灵特工风主阵容葫芦娃的阵容张家界诶我吗长治到成都东怎么坐火车小样就你矫情是什么歌77bbs金立s8摄像头碎了换个多钱太原到上海的火车票成都东到眉山东站的公交车绅士漫画h里番本子库张家界蹦极多少钱5s英雄联盟8月幸运召唤师网址女生左手食指带戒指是什么意思双鱼男爱上一个人长治到太原火车时刻表歌词小样就你矫情火影那个叫什么风暴来着个性签名长治到关林上海特产龙王传说最新章节2 4 8 16
柔和双沟女生中财网首页 财经http://pan.baidu.com/s/1plpjtn9周公解梦大全查询2345曹云金再讽郭德纲总裁大人行行好中财网第一财经传媒教师节全文男子砸毁15墓碑黄岩岛最新填海图引起的疲缘来未迟落跑甜心不好惹梁朝伟与替身同框笑傲江湖电视剧任贤齐小起名字女孩名字海运行李到堪培拉确定诱爱99天 司少的天价宝贝什么是遥控魔棒徽信表情动态搞笑图片教师节征文100字安微联通网上营业厅甜宠百分百:校草的萌萌未婚妻豪门重生之暖爱成婚nikehypershift和kd5哪个好看韭菜炒鸡蛋陈赫玩王者荣耀虎牙楚河三国演义小说txt下载威县欧派炒馍花怎么做好吃黄岩岛最新消息2016年中秋节诗句大全祝福教师节征文菜谱柔和双沟卖的怎

0.42888593673706055

In [16]:
res.head()

,Query_List
0,"[柔和, 双沟, 女生, 中财网, 首页, , 财经, http, pan, baidu,..."
1,"[广州, 厨宝, 烤箱, 世情, 薄, 人情, 恶, 雨送, 黄昏, 花易落, 晓, 风干,..."
2,"[钻石, 之泪, 耳机, 盘锦, 沈阳, 旅顺, 公交, 辽宁, 阜新, 车牌, baidu..."
3,"[受欢迎, 狗狗, 排行榜, 舶, 读, 场景, 描, , 写, 范例, 三维, 绘图, ..."
4,"[干槽症, 自愈, 太太, 万岁, 叶, 舒心, 美国, 干槽症, 右, 眼皮, 跳, 怎么..."


10条数据用0.18s,100000条数据大概要用1800s

In [19]:
res = train.loc[:,['Query_List']].applymap(preprocess)

In [20]:
res.head(3)

,Query_List
0,"[柔和, 双沟, \t, 女生, \t, 中财网, 首页, , 财经, \t, pan, ..."
1,"[广州, 厨宝, 烤箱, \t, 世情, 薄, ,, 人情, 恶, ,, 雨送, 黄昏, 花..."
2,"[钻石, 之泪, 耳机, \t, 盘锦, 到, 沈阳, \t, 旅顺, 公交, \t, 辽宁..."


In [35]:
train.loc[:,'Query_List1']=res.loc[:,'Query_List1']

In [36]:
train.head()

,ID,Age,Gender,Education,Query_List,Query_List1
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...,"[柔和, 双沟, 女生, 中财网, 首页, , 财经, pan, 周公, 解梦, 大全, ..."
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...","[广州, 厨宝, 烤箱, 世情, 薄, ,, 人情, 恶, ,, 雨送, 黄昏, 花易落, ..."
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...,"[钻石, 之泪, 耳机, 盘锦, 到, 沈阳, 旅顺, 公交, 辽宁, 阜新, 车牌, ba..."
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,"[最, 受欢迎, 狗狗, 排行榜, 舶, 怎么, 读, 场景, 描, , 写, 范例, 三..."
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,"[干槽症, 能, 自愈, 吗, 太太, 万岁, 叶, 舒心, 去, 没, 去, 美国, 干槽..."


#### 保存数据

In [63]:
#### 将数据保存下来

In [56]:
train.to_csv('train.csv')

In [71]:
train.loc[:,['Query_List1']] = train.loc[:,['Query_List1']].applymap(lambda x:" ".join(x))

In [39]:
train = pd.read_csv('train.csv')

In [40]:
train.head()

,Unnamed: 0,ID,Age,Gender,Education,Query_List1
0,0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,"['柔和', '双沟', '女生', '中财网', '首页', ' ', '财经', 'pa..."
1,1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"['广州', '厨宝', '烤箱', '世情', '薄', ',', '人情', '恶', ..."
2,2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,"['钻石', '之泪', '耳机', '盘锦', '到', '沈阳', '旅顺', '公交'..."
3,3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,"['最', '受欢迎', '狗狗', '排行榜', '舶', '怎么', '读', '场景'..."
4,4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,"['干槽症', '能', '自愈', '吗', '太太', '万岁', '叶', '舒心',..."


由于后面发现计算量比较大所以加了去停用词的程序

In [62]:
def dstopwords(data):
    data = eval(data)
    word_state = pd.DataFrame(data,columns=['segment'])
    data = ' '.join(word_state[~word_state['segment'].isin(stopwords.stopword)]['segment'].tolist())
    return data

In [59]:
dstopwords(eval(train.iloc[0,5]))

(986, 1)
804


AttributeError: 'str' object has no attribute 'shape'

In [63]:
res = train.loc[:,['Query_List1']].applymap(dstopwords)

In [71]:
train.loc[:,'Query_List']= res.Query_List1

In [72]:
train.head()

,Unnamed: 0,ID,Age,Gender,Education,Query_List1,Query_List
0,0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,"['柔和', '双沟', '女生', '中财网', '首页', ' ', '财经', 'pa...",柔和 双沟 女生 中财网 首页 财经 pan 周公 解梦 大全 查询 2345 曹云金 ...
1,1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"['广州', '厨宝', '烤箱', '世情', '薄', ',', '人情', '恶', ...",广州 厨宝 烤箱 世情 薄 人情 恶 雨送 黄昏 花易落 晓 风干 泪痕 厦门 酒店用品 批...
2,2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,"['钻石', '之泪', '耳机', '盘锦', '到', '沈阳', '旅顺', '公交'...",钻石 之泪 耳机 盘锦 沈阳 旅顺 公交 辽宁 阜新 车牌 baidu k715 k716 ...
3,3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,"['最', '受欢迎', '狗狗', '排行榜', '舶', '怎么', '读', '场景'...",受欢迎 狗狗 排行榜 舶 读 场景 描 写 范例 三维 绘图 软件 枣 酸奶 吃 壮观 ...
4,4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,"['干槽症', '能', '自愈', '吗', '太太', '万岁', '叶', '舒心',...",干槽症 自愈 太太 万岁 叶 舒心 美国 干槽症 右 眼皮 跳 怎么回事 麦当劳 麦 旋风 ...


In [73]:
train1 = train.loc[:,['ID','Age','Gender','Education','Query_List']]
train.to_csv('train1.csv')

In [3]:
train = pd.read_csv('train1.csv')

In [81]:
train.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'ID', 'Age', 'Gender', 'Education',
       'Query_List1', 'Query_List'],
      dtype='object')

In [4]:
train.drop(['Unnamed: 0', 'Unnamed: 0.1','Query_List1'],axis=1)

,ID,Age,Gender,Education,Query_List
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和 双沟 女生 中财网 首页 财经 pan 周公 解梦 大全 查询 2345 曹云金 ...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,广州 厨宝 烤箱 世情 薄 人情 恶 雨送 黄昏 花易落 晓 风干 泪痕 厦门 酒店用品 批...
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石 之泪 耳机 盘锦 沈阳 旅顺 公交 辽宁 阜新 车牌 baidu k715 k716 ...
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,受欢迎 狗狗 排行榜 舶 读 场景 描 写 范例 三维 绘图 软件 枣 酸奶 吃 壮观 ...
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症 自愈 太太 万岁 叶 舒心 美国 干槽症 右 眼皮 跳 怎么回事 麦当劳 麦 旋风 ...
...,...,...,...,...,...
99995,E797FFCDCAF3899AB4D17B61170D8BFF,1,1,5,梦 三生 逆行 天后 漫画 结局 英雄 联盟 角色 介绍 大全 韩剧 网 最新 韩国 电视剧...
99996,E06375F7D092ABDE78C2D79E4725D6B0,1,1,5,中国 军队 配枪 女生 央视 版权 重庆 一次性 手术刀 精神枷锁 检察官 舆论 精神 禁锢...
99997,D55119CB0B9366B20974522B58C00912,2,2,5,英文翻译 幼儿园 面试 讲课 学前教育 书 版 shock 幼儿园 教师 资格证 面试 ...
99998,EB4DBBD602C6459A19A77F09035E170C,3,2,3,哈尔滨 祖研 中 医院地址 指甲盖 侧面 肉 白色 补 骨质 副作用 哈尔滨 呼兰 黑龙江 ...


#### 词向量化

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(
    analyzer='word', # tokenise by character ngrams
    max_features=4000,  # keep the most common 4000 ngrams
    ngram_range = (1,4)
)
vec.fit(train.Query_List.tolist())

MemoryError: 

#### 保存模型

In [4]:
path = 'vectorizer.model'
dump(vec, path)

NameError: name 'dump' is not defined

In [5]:
path = 'vectorizer.model'
vec = load(path)

NameError: name 'load' is not defined

#### 将数据分为训练集和测试集

In [ ]:
res = vec.transform([' '.join(train.iloc[3,4])])

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
train[train['Age']>0].Age.tolist()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(train[train['Age']>0].Query_List1.tolist(),\
                                                train[train['Age']>0].Age.tolist())


#### 建立模型


In [ ]:
# 对年龄建立预测模型
from sklearn.naive_bayes import MultinomialNB
age_classifier = MultinomialNB()
age_classifier.fit(vec.transform(X_train), y_train)

#### 检验模型

In [ ]:
age_classifier.score(vec.transform(X_test), y_test)

In [ ]:
age_classifier.score(vec.transform(X_train), y_train)

#### 同样方法对性别和学历进行分析

In [ ]:
# 对性别建立预测模型
X_train,X_test,y_train,y_test = train_test_split(train[train['Gender']>0].Query_List1.tolist(),\
                                                train[train['Gender']>0].Gender.tolist())
gender_classifier = MultinomialNB()
gender_classifier.fit(vec.transform(X_train), y_train)
gender_classifier.score(vec.transform(X_test), y_test)

In [ ]:
# 对学历建立预测模型
X_train,X_test,y_train,y_test = train_test_split(train[train['Education']>0].Query_List1.tolist(),\
                                                train[train['Education']>0].Education.tolist())
education_classifier = MultinomialNB()
education_classifier.fit(vec.transform(X_train), y_train)
education_classifier.score(vec.transform(X_train), y_train)

In [ ]:
train[train['Education']>0].Query_List1.tolist()

In [ ]:
print(vec.transform([X_test[0]]))